# Table of Contents

&emsp;[<a id="#vocab" /> <font color='yellow' size=5> Vocab </font>](#%3Ca-id%3D%22%23vocab%22-/%3E-%3Cfont-color%3D%27yellow%27%3E-Vocab-%3C/font%3E)<br>
&emsp;&emsp;[1. Comparison](#1.-Comparison)<br>
&emsp;&emsp;[2. Intensifier](#2.-Intensifier)<br><br>
&emsp;[<a id="Taxonomy" /> <font color='yellow' size=5> Taxonomy </font>](#%3Ca-id%3D%22Taxonomy%22-/%3E-%3Cfont-color%3D%27yellow%27%3E-Taxonomy-%3C/font%3E)<br>
&emsp;&emsp;[1. Size, Shape, Color, Age, Material](#1.-Size%2C-Shape%2C-Color%2C-Age%2C-Material)<br>
&emsp;&emsp;[2. Professions vs nationalities](#2.-Professions-vs-nationalities)<br>
&emsp;&emsp;[3. Animal vs Vehicles - Example1](#3.-Animal-vs-Vehicles---Example1)<br>
&emsp;&emsp;[4. Animal vs Vehicles - Example2](#4.-Animal-vs-Vehicles---Example2)<br>
&emsp;&emsp;[5. Synonyms](#5.-Synonyms)<br>
&emsp;&emsp;[6. Antonyms](#6.-Antonyms)<br>
&emsp;&emsp;[7. Antonyms Comparison](#7.-Antonyms-Comparison)<br><br>
&emsp;[<a id="Temporal" /> <font color='yellow' size=5> Temporal </font>](#%3Ca-id%3D%22Temporal%22-/%3E-%3Cfont-color%3D%27yellow%27%3E-Temporal-%3C/font%3E)<br>
&emsp;&emsp;[1. Change in profession](#1.-Change-in-profession)<br>
&emsp;&emsp;[2. Understand time difference](#2.-Understand-time-difference)<br><br>
&emsp;[<a id="Negation" /> <font color='yellow' size=5> Negation </font>](#%3Ca-id%3D%22Negation%22-/%3E-%3Cfont-color%3D%27yellow%27%3E-Negation-%3C/font%3E)<br>
&emsp;&emsp;[1. Negation in Context](#1.-Negation-in-Context)<br>
&emsp;&emsp;[3. Negation in Question](#3.-Negation-in-Question)<br><br>
&emsp;[<a id="Coref" /> <font color='yellow' size=5> Coref </font>](#%3Ca-id%3D%22Coref%22-/%3E-%3Cfont-color%3D%27yellow%27%3E-Coref-%3C/font%3E)<br>
&emsp;&emsp;[1. He/she coref](#1.-He/she-coref)<br>
&emsp;&emsp;[2. His/Her coref](#2.-His/Her-coref)<br>
&emsp;&emsp;[3. Former and Latter](#3.-Former-and-Latter)<br>

In [92]:
%load_ext autoreload
%autoreload 2
import checklist.editor
import munch
import itertools
from checklist.test_suite import TestSuite
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
from checklist.perturb import Perturb
from helper import get_finetuned_electra_predictor, format_squad_with_context,show_example, crossproduct, export_suite_to_jsonl, get_summary,display_and_export_mdtable

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
suite = TestSuite()
editor = checklist.editor.Editor()
model_predictor = get_finetuned_electra_predictor()

/opt/anaconda3/envs/nlp-proj/lib/python3.10/site-packages/iso639/iso639.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


## <a id="#vocab" /> <font color='yellow'> Vocab </font>

### 1. Comparison

In [3]:
### <a id="1-comparison"></a> 1. Comparisonart', 'tall', 'young', 'strong', 'short', 'tough', 'cool', 'fast', 'nice', 'small', 'dark', 'wise', 'rich', 'great', 'weak', 'high', 'slow', 'strange', 'clean']
adj = [(x.rstrip('e'), x) for x in adj]


t_comparison = editor.template(
    [(
    '{first_name} is {adj[0]}er than {first_name1}.',
    'Who is less {adj[1]}?'
    ),(
    '{first_name} is {adj[0]}er than {first_name1}.',
    'Who is {adj[0]}er?'
    ),
    ],
    labels = ['{first_name1}','{first_name}'],
    adj=adj,
    remove_duplicates=True,
    nsamples=500,
    save=True
)

In [4]:
show_example(t_comparison, n=1)

  [1] ('Jonathan is stranger than Carolyn.', 'Who is less strange?') , pred: Jonathan , label: Carolyn
  [2] ('Jonathan is stranger than Carolyn.', 'Who is stranger?') , pred: Jonathan , label: Jonathan
---


In [5]:
name = 'Comparisons'
description = 'A is COMP than B. Who is more / less COMP?'
test = MFT(**t_comparison, name=name, description=description, capability='Vocabulary')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 996 examples
Test cases:      498
Fails (rate):    498 (100.0%)

Example fails:
C: Jack is tougher than Greg.
Q: Who is less tough?
A: Greg
P: Jack


----


### 2. Intensifier

In [6]:
state = editor.suggest('John is very {mask} about the project.')[:20]
very = ['very', 'extremely', 'really', 'quite', 'incredibly', 'particularly', 'highly', 'super']
somewhat = ['a little', 'somewhat', 'slightly', 'mildly']

t_intensifier = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {very} {s} about the project. {first_name1} is {s} about the project.',
            '{first_name1} is {s} about the project. {first_name} is {very} {s} about the project.',
            '{first_name} is {s} about the project. {first_name1} is {somewhat} {s} about the project.',
            '{first_name1} is {somewhat} {s} about the project. {first_name} is {s} about the project.',
            '{first_name} is {very} {s} about the project. {first_name1} is {somewhat} {s} about the project.',
            '{first_name1} is {somewhat} {s} about the project. {first_name} is {very} {s} about the project.',
        ],
        'qas': [
            (
                'Who is most {s} about the project?',
                '{first_name}'
            ), 
            (
                'Who is least {s} about the project?',
                '{first_name1}'
            ), 
            
        ]
        
    },
    s = state,
    very=very,
    somewhat=somewhat,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))

/opt/anaconda3/envs/nlp-proj/lib/python3.10/site-packages/checklist/text_generation.py:171: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:256.)
  to_pred = torch.tensor(to_pred, device=self.device).to(torch.int64)


In [7]:
show_example(t_intensifier)

  [1] ('Kathy is incredibly upbeat about the project. Jimmy is upbeat about the project.', 'Who is most upbeat about the project?') , pred: Jimmy , label: Kathy
  [2] ('Kathy is incredibly upbeat about the project. Jimmy is upbeat about the project.', 'Who is least upbeat about the project?') , pred: Jimmy , label: Jimmy
  [3] ('Jimmy is upbeat about the project. Kathy is incredibly upbeat about the project.', 'Who is most upbeat about the project?') , pred: Kathy , label: Kathy
  [4] ('Jimmy is upbeat about the project. Kathy is incredibly upbeat about the project.', 'Who is least upbeat about the project?') , pred: Kathy , label: Jimmy
  [5] ('Kathy is upbeat about the project. Jimmy is somewhat upbeat about the project.', 'Who is most upbeat about the project?') , pred: Jimmy , label: Kathy
  [6] ('Kathy is upbeat about the project. Jimmy is somewhat upbeat about the project.', 'Who is least upbeat about the project?') , pred: Jimmy , label: Jimmy
  [7] ('Jimmy is somewhat upbeat ab

In [8]:
name = 'Intensifiers'
desc = '(very, super, extremely) and reducers (somewhat, kinda, etc)?'
test = MFT(**t_intensifier, name=name, description=desc, capability='Vocabulary')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context, n_per_testcase=2)
suite.add(test)

Predicting 5988 examples
Test cases:      499
Fails (rate):    499 (100.0%)

Example fails:
C: George is incredibly particular about the project. Edward is particular about the project.
Q: Who is most particular about the project?
A: George
P: Edward

C: Edward is particular about the project. George is incredibly particular about the project.
Q: Who is least particular about the project?
A: Edward
P: George


----


## <a id="Taxonomy" /> <font color='yellow'> Taxonomy </font>

### 1. Size, Shape, Color, Age, Material

In [9]:
order = ['size', 'shape', 'age', 'color']
props = []
properties = {
    'color' : ['red', 'blue','yellow', 'green', 'pink', 'white', 'black', 'orange', 'grey', 'purple', 'brown'],
    'size' : ['big', 'small', 'tiny', 'enormous'],
    'age' : ['old', 'new'],
    'shape' : ['round', 'oval', 'square', 'triangular'],
    'material' : ['iron', 'wooden', 'ceramic', 'glass', 'stone']
}
objects = ['box', 'clock', 'table', 'object', 'toy', 'painting', 'sculpture', 'thing', 'figure']

for i in range(len(order)):
    for j in range(i + 1, len(order)):
        p1, p2 = order[i], order[j]
        for v1, v2 in itertools.product(properties[p1], properties[p2]):
            props.append(munch.Munch({
                'p1': p1,
                'p2': p2,
                'v1': v1,
                'v2': v2,
            }))
            
t_properties = crossproduct(editor.template(
    {
        'contexts': [
            'There is {a:p.v1} {p.v2} {obj} in the room.',
            'There is {a:obj} in the room. The {obj} is {p.v1} and {p.v2}.',
        ],
        'qas': [
            (
                'What {p.p1} is the {obj}?',
                '{p.v1}'
            ), 
            (
                'What {p.p2} is the {obj}?',
                '{p.v2}'
            ), 
            
        ]
    },
    obj=objects,
    p=props,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))


In [10]:
show_example(t_properties, n=2)

  [1] ('There is a new purple box in the room.', 'What age is the box?') , pred: purple , label: new
  [2] ('There is a new purple box in the room.', 'What color is the box?') , pred: purple , label: purple
  [3] ('There is a box in the room. The box is new and purple.', 'What age is the box?') , pred: new and purple , label: new
  [4] ('There is a box in the room. The box is new and purple.', 'What color is the box?') , pred: purple , label: purple
---
  [1] ('There is a triangular new table in the room.', 'What shape is the table?') , pred: triangular , label: triangular
  [2] ('There is a triangular new table in the room.', 'What age is the table?') , pred: triangular new , label: new
  [3] ('There is a table in the room. The table is triangular and new.', 'What shape is the table?') , pred: triangular , label: triangular
  [4] ('There is a table in the room. The table is triangular and new.', 'What age is the table?') , pred: triangular and new , label: new
---


In [11]:
name = 'Properites'
desc = 'size, shape, age, color'
test = MFT(**t_properties, name=name, description=desc, capability='Taxonomy')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 2000 examples
Test cases:      500
Fails (rate):    499 (99.8%)

Example fails:
C: There is an enormous triangular object in the room.
Q: What size is the object?
A: enormous
P: triangular

C: There is an object in the room. The object is enormous and triangular.
Q: What size is the object?
A: enormous
P: enormous and triangular


----


### 2. Professions vs nationalities

In [12]:
professions = editor.suggest('{first_name} works as {a:mask}.')[:30]
professions += editor.suggest('{first_name} {last_name} works as {a:mask}.')[:30]
professions = list(set(professions))
if 'translator' in professions:
    professions.remove('translator')


def clean(string):
    return string.lstrip('[a,the,an,in,at] ').rstrip('.')
def expect_squad(x, pred, conf, label=None, meta=None):
    return clean(pred) == clean(label)
expect_squad = Expect.single(expect_squad)


t_profession_nationtionality = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {a:nat} {prof}.',
            '{first_name} is {a:prof}. {first_name} is {nat}.',
            '{first_name} is {nat}. {first_name} is {a:prof}.',
            '{first_name} is {nat} and {a:prof}.',
            '{first_name} is {a:prof} and {nat}.',
        ],
        'qas': [
            (
                'What is {first_name}\'s job?',
                '{prof}'
            ), 
            (
                'What is {first_name}\'s nationality?',
                '{nat}'
            ), 
            
        ]
        
    },
    nat = editor.lexicons['nationality'][:10],
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    save=True,
    ))

In [13]:
show_example(t_profession_nationtionality, n=1)

  [1] ('Janet is an American investor.', "What is Janet's job?") , pred: American investor , label: investor
  [2] ('Janet is an American investor.', "What is Janet's nationality?") , pred: American , label: American
  [3] ('Janet is an investor. Janet is American.', "What is Janet's job?") , pred: investor , label: investor
  [4] ('Janet is an investor. Janet is American.', "What is Janet's nationality?") , pred: American , label: American
  [5] ('Janet is American. Janet is an investor.', "What is Janet's job?") , pred: investor , label: investor
  [6] ('Janet is American. Janet is an investor.', "What is Janet's nationality?") , pred: American , label: American
  [7] ('Janet is American and an investor.', "What is Janet's job?") , pred: American and an investor , label: investor
  [8] ('Janet is American and an investor.', "What is Janet's nationality?") , pred: American , label: American
  [9] ('Janet is an investor and American.', "What is Janet's job?") , pred: investor and Ameri

In [14]:
name = 'Profession vs nationality'
test = MFT(**t_profession_nationtionality, name=name, expect=expect_squad, description='',  capability='Taxonomy')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 5000 examples
Test cases:      500
Fails (rate):    260 (52.0%)

Example fails:
C: Donald is an educator and Nigerian.
Q: What is Donald's job?
A: educator
P: educator and Nigerian


----


### 3. Animal vs Vehicles - Example1

In [15]:
animals = ['dog', 'cat', 'bull', 'cow', 'fish', 'serpent', 'snake', 'lizard', 'hamster', 'rabbit', 'guinea pig', 'iguana', 'duck']
vehicles = ['car', 'truck', 'train', 'motorcycle', 'bike', 'firetruck', 'tractor', 'van', 'SUV', 'minivan']
t_animal_vehicles = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} has {a:animal} and {a:vehicle}.',
            '{first_name} has {a:vehicle} and {a:animal}.',
        ],
        'qas': [
            (
                'What animal does {first_name} have?',
                '{animal}'
            ), 
            (
                'What vehicle does {first_name} have?',
                '{vehicle}'
            ), 
            
        ]
        
    },
    animal=animals,
    vehicle=vehicles,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))

In [16]:
show_example(t_animal_vehicles)

  [1] ('Mike has a hamster and a tractor.', 'What animal does Mike have?') , pred: hamster , label: hamster
  [2] ('Mike has a hamster and a tractor.', 'What vehicle does Mike have?') , pred: hamster and a tractor , label: tractor
  [3] ('Mike has a tractor and a hamster.', 'What animal does Mike have?') , pred: a tractor and a hamster , label: hamster
  [4] ('Mike has a tractor and a hamster.', 'What vehicle does Mike have?') , pred: a tractor , label: tractor
---


In [17]:
name = 'Animal vs Vehicle'
test = MFT(**t_animal_vehicles, name=name, description='', capability='Taxonomy', expect=expect_squad)
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test, overwrite=True)

Predicting 2000 examples
Test cases:      500
Fails (rate):    495 (99.0%)

Example fails:
C: Alex has a serpent and a bike.
Q: What vehicle does Alex have?
A: bike
P: a serpent and a bike


----


### 4. Animal vs Vehicles - Example2

In [18]:
animals = ['dog', 'cat', 'bull', 'cow', 'fish', 'serpent', 'snake', 'lizard', 'hamster', 'rabbit', 'guinea pig', 'iguana', 'duck']
vehicles = ['car', 'truck', 'train', 'motorcycle', 'bike', 'firetruck', 'tractor', 'van', 'SUV', 'minivan']
t_animals_vehicles_2 = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} bought {a:animal}. {first_name2} bought {a:vehicle}.',
            '{first_name2} bought {a:vehicle}. {first_name} bought {a:animal}.',
        ],
        'qas': [
            (
                'Who bought an animal?',
                '{first_name}'
            ), 
            (
                'Who bought a vehicle?',
                '{first_name2}'
            ), 
            
        ]
        
    },
    animal=animals,
    vehicle=vehicles,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))


In [19]:
show_example(t_animals_vehicles_2)

  [1] ('Sandra bought a cat. Ellen bought a tractor.', 'Who bought an animal?') , pred: Sandra , label: Sandra
  [2] ('Sandra bought a cat. Ellen bought a tractor.', 'Who bought a vehicle?') , pred: Ellen , label: Ellen
  [3] ('Ellen bought a tractor. Sandra bought a cat.', 'Who bought an animal?') , pred: Sandra , label: Sandra
  [4] ('Ellen bought a tractor. Sandra bought a cat.', 'Who bought a vehicle?') , pred: Ellen , label: Ellen
---


In [20]:

name = 'Animal vs Vehicle v2'
test = MFT(**t_animals_vehicles_2, name=name, description='', capability='Taxonomy', expect=expect_squad)
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test, overwrite=True)

Predicting 1996 examples
Test cases:      499
Fails (rate):    266 (53.3%)

Example fails:
C: Anne bought a duck. Harriet bought a train.
Q: Who bought an animal?
A: Anne
P: Harriet


----


### 5. Synonyms 

In [21]:
synonyms = [ ('spiritual', 'religious'), ('angry', 'furious'), ('organized', 'organised'),
            ('vocal', 'outspoken'), ('grateful', 'thankful'), ('intelligent', 'smart'),
            ('humble', 'modest'), ('courageous', 'brave'), ('happy', 'joyful'), ('scared', 'frightened'),
           ]

t_synonym = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is very {s1[0]}. {first_name2} is very {s2[0]}.',
            '{first_name2} is very {s2[0]}. {first_name} is very {s1[0]}.',
        ],
        'qas': [
            (
                'Who is {s1[1]}?',
                '{first_name}'
            ), 
            (
                'Who is {s2[1]}?',
                '{first_name2}'
            ), 
            
        ]
        
    },
    s=synonyms,
    remove_duplicates=True,
    nsamples=250,
    save=True
   ))
t_synonym += crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is very {s1[1]}. {first_name2} is very {s2[1]}.',
            '{first_name2} is very {s2[1]}. {first_name} is very {s1[1]}.',
        ],
        'qas': [
            (
                'Who is {s1[0]}?',
                '{first_name}'
            ), 
            (
                'Who is {s2[0]}?',
                '{first_name2}'
            ), 
            
        ]
        
    },
    s=synonyms,
    remove_duplicates=True,
    nsamples=250,
    save=True
    )) 

In [22]:
show_example(t_synonym)

  [1] ('Kathleen is very vocal. Edwin is very angry.', 'Who is outspoken?') , pred: Edwin , label: Kathleen
  [2] ('Kathleen is very vocal. Edwin is very angry.', 'Who is furious?') , pred: Edwin , label: Edwin
  [3] ('Edwin is very angry. Kathleen is very vocal.', 'Who is outspoken?') , pred: Kathleen , label: Kathleen
  [4] ('Edwin is very angry. Kathleen is very vocal.', 'Who is furious?') , pred: Edwin , label: Edwin
---


In [23]:
name = 'Synonyms'
test = MFT(**t_synonym, name=name, description='', capability='Taxonomy', expect=expect_squad)
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 1832 examples
Test cases:      458
Fails (rate):    21 (4.6%)

Example fails:
C: Sam is very happy. Eric is very grateful.
Q: Who is joyful?
A: Sam
P: Eric


----


### 6. Antonyms

In [24]:
comp_pairs = [('better', 'worse'), ('older', 'younger'), ('smarter', 'dumber'), ('taller', 'shorter'), ('bigger', 'smaller'), ('stronger', 'weaker'), ('faster', 'slower'), ('darker', 'lighter'), ('richer', 'poorer'), ('happier', 'sadder'), ('louder', 'quieter'), ('warmer', 'colder')]
comp_pairs = list(set(comp_pairs))

In [25]:
t_antonymns = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {comp[0]} than {first_name1}.',
            '{first_name1} is {comp[1]} than {first_name}.',
        ],
        'qas': [
            (
                'Who is {comp[1]}?',
                '{first_name1}',
            ),
            (
                'Who is {comp[0]}?',
                '{first_name}',
            )
            
        ]
        ,
    },
    comp=comp_pairs,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))

In [26]:
show_example(t_antonymns)

  [1] ('Al is louder than Gary.', 'Who is quieter?') , pred: Al , label: Gary
  [2] ('Al is louder than Gary.', 'Who is louder?') , pred: Al , label: Al
  [3] ('Gary is quieter than Al.', 'Who is quieter?') , pred: Gary , label: Gary
  [4] ('Gary is quieter than Al.', 'Who is louder?') , pred: Gary , label: Al
---


In [27]:
name = 'Antonyms'
test = MFT(**t_antonymns, name=name, description='A is COMP than B. Who is antonym(COMP)? B', capability='Taxonomy')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 1992 examples
Test cases:      498
Fails (rate):    498 (100.0%)

Example fails:
C: Francis is darker than Bruce.
Q: Who is lighter?
A: Bruce
P: Francis

C: Bruce is lighter than Francis.
Q: Who is darker?
A: Francis
P: Bruce


----


### 7. Antonyms Comparison

In [28]:
antonym_adjs = [('progressive', 'conservative'),('religious', 'secular'),('positive', 'negative'),('defensive', 'offensive'),('rude',  'polite'),('optimistic', 'pessimistic'),('stupid', 'smart'),('negative', 'positive'),('unhappy', 'happy'),('active', 'passive'),('impatient', 'patient'),('powerless', 'powerful'),('visible', 'invisible'),('fat', 'thin'),('bad', 'good'),('cautious', 'brave'), ('hopeful', 'hopeless'),('insecure', 'secure'),('humble', 'proud'),('passive', 'active'),('dependent', 'independent'),('pessimistic', 'optimistic'),('irresponsible', 'responsible'),('courageous', 'fearful')]

t_antonymns_compare = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is more {a[0]} than {first_name1}.',
            '{first_name1} is more {a[1]} than {first_name}.',
            '{first_name} is less {a[1]} than {first_name1}.',
            '{first_name1} is less {a[0]} than {first_name}.',
        ],
        'qas': [
            (
                'Who is more {a[0]}?',
                '{first_name}',
            ),
            (
                'Who is less {a[0]}?',
                '{first_name1}',
            ),
            (
                'Who is more {a[1]}?',
                '{first_name1}',
            ),
            (
                'Who is less {a[1]}?',
                '{first_name}',
            ),
        ]
        ,
    },
    a = antonym_adjs,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))

In [29]:
show_example(t_antonymns_compare)

  [1] ('Lauren is more active than Arthur.', 'Who is more active?') , pred: Lauren , label: Lauren
  [2] ('Lauren is more active than Arthur.', 'Who is less active?') , pred: Lauren , label: Arthur
  [3] ('Lauren is more active than Arthur.', 'Who is more passive?') , pred: Lauren , label: Arthur
  [4] ('Lauren is more active than Arthur.', 'Who is less passive?') , pred: Lauren , label: Lauren
  [5] ('Arthur is more passive than Lauren.', 'Who is more active?') , pred: Arthur , label: Lauren
  [6] ('Arthur is more passive than Lauren.', 'Who is less active?') , pred: Arthur , label: Arthur
  [7] ('Arthur is more passive than Lauren.', 'Who is more passive?') , pred: Arthur , label: Arthur
  [8] ('Arthur is more passive than Lauren.', 'Who is less passive?') , pred: Arthur , label: Lauren
  [9] ('Lauren is less passive than Arthur.', 'Who is more active?') , pred: Lauren , label: Lauren
  [10] ('Lauren is less passive than Arthur.', 'Who is less active?') , pred: Lauren , label: Arthur

In [30]:
description = 'A is more X than B. Who is more antonym(X)? B. Who is less X? B. Who is more X? A. Who is less antonym(X)? A.'
name= 'Antonyms Comparison'
test = MFT(**t_antonymns_compare, name=name, description=description, capability='Taxonomy')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 7984 examples
Test cases:      499
Fails (rate):    499 (100.0%)

Example fails:
C: Suzanne is more irresponsible than Tom.
Q: Who is less irresponsible?
A: Tom
P: Suzanne

C: Suzanne is more irresponsible than Tom.
Q: Who is more responsible?
A: Tom
P: Suzanne

C: Tom is more responsible than Suzanne.
Q: Who is more irresponsible?
A: Suzanne
P: Tom


----


## <a id="Temporal" /> <font color='yellow'> Temporal </font>

### 1. Change in profession

In [32]:
t_profession = crossproduct(editor.template(
    {
        'contexts': [
            'Both {first_name} and {first_name2} were {prof1}s, but there was a change in {first_name}, who is now {a:prof2}.',
            'Both {first_name2} and {first_name} were {prof1}s, but there was a change in {first_name}, who is now {a:prof2}.',
        ],
        'qas': [
            (
                'Who is {a:prof2}?',
                '{first_name}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))


In [33]:
show_example(t_profession)

  [1] ('Both Evelyn and Charlie were nurses, but there was a change in Evelyn, who is now an assistant.', 'Who is an assistant?') , pred: Evelyn , label: Evelyn
  [2] ('Both Charlie and Evelyn were nurses, but there was a change in Evelyn, who is now an assistant.', 'Who is an assistant?') , pred: Evelyn , label: Evelyn
---


In [34]:
name = 'There was a change in profession'
test = MFT(**t_profession, expect=expect_squad, capability='Temporal', name=name, description='' )
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 970 examples
Test cases:      485
Fails (rate):    10 (2.1%)

Example fails:
C: Both Arthur and Don were historians, but there was a change in Don, who is now an artist.
Q: Who is an artist?
A: Don
P: Arthur and Don


----


### 2. Understand time difference
e.g before, after

In [35]:
t_time_difference = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} became a {prof} before {first_name2} did.',
            '{first_name2} became a {prof} after {first_name} did.',
        ],
        'qas': [
            (
                'Who became a {prof} first?',
                '{first_name}'
            ), 
            (
                'Who became a {prof} last?',
                '{first_name2}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))

In [36]:
show_example(t_time_difference)

  [1] ('Lucy became a artist before Kate did.', 'Who became a artist first?') , pred: Lucy , label: Lucy
  [2] ('Lucy became a artist before Kate did.', 'Who became a artist last?') , pred: Lucy , label: Kate
  [3] ('Kate became a artist after Lucy did.', 'Who became a artist first?') , pred: Kate , label: Lucy
  [4] ('Kate became a artist after Lucy did.', 'Who became a artist last?') , pred: empty , label: Kate
---


In [37]:

description = 'Understanding before / after -> first / last.'
name="Time Difference"
test = MFT(**t_time_difference, expect=expect_squad, capability='Temporal', name=name, description=description )
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 1988 examples
Test cases:      497
Fails (rate):    497 (100.0%)

Example fails:
C: Lisa became a actor before Mike did.
Q: Who became a actor last?
A: Mike
P: Lisa

C: Mike became a actor after Lisa did.
Q: Who became a actor first?
A: Lisa
P: empty

C: Mike became a actor after Lisa did.
Q: Who became a actor last?
A: Mike
P: Lisa


----


## <a id="Negation" /> <font color='yellow'> Negation </font>

### 1. Negation in Context

In [38]:
t_context_negation = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is not {a:prof}. {first_name2} is.',
            '{first_name2} is {a:prof}. {first_name} is not.',
        ],
        'qas': [
            (
                'Who is {a:prof}?',
                '{first_name2}'
            ), 
            (
                'Who is not {a:prof}?',
                '{first_name}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))

In [39]:
show_example(t_context_negation)

  [1] ('Bob is not an accountant. Helen is.', 'Who is an accountant?') , pred: Bob , label: Helen
  [2] ('Bob is not an accountant. Helen is.', 'Who is not an accountant?') , pred: Bob , label: Bob
  [3] ('Helen is an accountant. Bob is not.', 'Who is an accountant?') , pred: Helen , label: Helen
  [4] ('Helen is an accountant. Bob is not.', 'Who is not an accountant?') , pred: Bob , label: Bob
---


In [40]:

name = 'Negation in context, may or may not be in question'
test = MFT(**t_context_negation, expect=expect_squad, capability='Negation', name=name, description='' )
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 1980 examples
Test cases:      495
Fails (rate):    487 (98.4%)

Example fails:
C: Robin is not an investor. Tony is.
Q: Who is an investor?
A: Tony
P: Robin


----


### 3. Negation in Question

In [41]:
t_question_negation = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {a:prof}. {first_name2} is {a:prof2}.',
            '{first_name2} is {a:prof2}. {first_name} is {a:prof}.',
        ],
        'qas': [
            (
                'Who is {a:prof}?',
                '{first_name}'
            ), 
            (
                'Who is not {a:prof}?',
                '{first_name2}'
            ), 
            (
                'Who is {a:prof2}?',
                '{first_name2}'
            ), 
            (
                'Who is not {a:prof2}?',
                '{first_name}'
            ), 
        ]
        
    },
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))


In [42]:

show_example(t_question_negation)

  [1] ('Amanda is an administrator. Jill is an organizer.', 'Who is an administrator?') , pred: Amanda , label: Amanda
  [2] ('Amanda is an administrator. Jill is an organizer.', 'Who is not an administrator?') , pred: Amanda , label: Jill
  [3] ('Amanda is an administrator. Jill is an organizer.', 'Who is an organizer?') , pred: Jill , label: Jill
  [4] ('Amanda is an administrator. Jill is an organizer.', 'Who is not an organizer?') , pred: Jill , label: Amanda
  [5] ('Jill is an organizer. Amanda is an administrator.', 'Who is an administrator?') , pred: Amanda , label: Amanda
  [6] ('Jill is an organizer. Amanda is an administrator.', 'Who is not an administrator?') , pred: Amanda , label: Jill
  [7] ('Jill is an organizer. Amanda is an administrator.', 'Who is an organizer?') , pred: Jill , label: Jill
  [8] ('Jill is an organizer. Amanda is an administrator.', 'Who is not an organizer?') , pred: Jill , label: Amanda
---


In [43]:
name = 'Negation in question only.'
test = MFT(**t_question_negation, expect=expect_squad, capability='Negation', name=name, description='' )
test.run(model_predictor)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 3912 examples
Test cases:      489
Fails (rate):    489 (100.0%)

Example fails:
C: Walter is an entrepreneur. Frances is an architect.
Q: Who is not an entrepreneur?
A: Frances
P: Walter

C: Walter is an entrepreneur. Frances is an architect.
Q: Who is not an architect?
A: Walter
P: Frances

C: Frances is an architect. Walter is an entrepreneur.
Q: Who is not an entrepreneur?
A: Frances
P: Walter


----
C: Jonathan is a model. Richard is a journalist.
Q: Who is not a model?
A: Richard
P: Jonathan

C: Jonathan is a model. Richard is a journalist.
Q: Who is not a journalist?
A: Jonathan
P: Richard

C: Richard is a journalist. Jonathan is a model.
Q: Who is not a model?
A: Richard
P: Jonathan


----
C: Gary is an entrepreneur. Larry is a waitress.
Q: Who is not an entrepreneur?
A: Larry
P: Gary

C: Gary is an entrepreneur. Larry is a waitress.
Q: Who is not a waitress?
A: Gary
P: Larry

C: Larry is a waitress. Gary is an entrepreneur.
Q: Who is not an entrepreneur?
A: Larry
P:

## <a id="Coref" /> <font color='yellow'> Coref </font>

### 1. He/she coref

In [44]:
if 'actress' in professions:
    professions.remove('actress')

t_he_she_coref = crossproduct(editor.template(
    {
        'contexts': [
            '{male} and {female} are friends. He is {a:prof1}, and she is {a:prof2}.',
            '{female} and {male} are friends. He is {a:prof1}, and she is {a:prof2}.',
            '{male} and {female} are friends. She is {a:prof2}, and he is {a:prof1}.',
            '{female} and {male} are friends. She is {a:prof2}, and he is {a:prof1}.',
        ],
        'qas': [
            (
                'Who is {a:prof1}?',
                '{male}'
            ), 
            (
                'Who is {a:prof2}?',
                '{female}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))


In [45]:
show_example(t_he_she_coref)

  [1] ('Donald and Emily are friends. He is an editor, and she is an executive.', 'Who is an editor?') , pred: Donald and Emily , label: Donald
  [2] ('Donald and Emily are friends. He is an editor, and she is an executive.', 'Who is an executive?') , pred: Donald and Emily , label: Emily
  [3] ('Emily and Donald are friends. He is an editor, and she is an executive.', 'Who is an editor?') , pred: Emily and Donald , label: Donald
  [4] ('Emily and Donald are friends. He is an editor, and she is an executive.', 'Who is an executive?') , pred: Emily and Donald , label: Emily
  [5] ('Donald and Emily are friends. She is an executive, and he is an editor.', 'Who is an editor?') , pred: Donald and Emily , label: Donald
  [6] ('Donald and Emily are friends. She is an executive, and he is an editor.', 'Who is an executive?') , pred: Donald and Emily , label: Emily
  [7] ('Emily and Donald are friends. She is an executive, and he is an editor.', 'Who is an editor?') , pred: Emily and Donald , 

In [46]:
name = 'Basic coref, he / she'
test = MFT(**t_he_she_coref, expect=expect_squad, name=name, description='', capability='Coref')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 3808 examples
Test cases:      476
Fails (rate):    476 (100.0%)

Example fails:
C: Ian and Stephanie are friends. He is a producer, and she is an editor.
Q: Who is a producer?
A: Ian
P: Ian and Stephanie

C: Ian and Stephanie are friends. He is a producer, and she is an editor.
Q: Who is an editor?
A: Stephanie
P: Ian and Stephanie

C: Stephanie and Ian are friends. He is a producer, and she is an editor.
Q: Who is a producer?
A: Ian
P: Stephanie and Ian


----


### 2. His/Her coref

In [47]:
t_his_her = crossproduct(editor.template(
    {
        'contexts': [
            '{male} and {female} are friends. His mom is {a:prof}.',
            '{female} and {male} are friends. His mom is {a:prof}.',
        ],
        'qas': [
            (
                'Whose mom is {a:prof}?',
                '{male}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=250,
    ))
t_his_her += crossproduct(editor.template(
    {
        'contexts': [
            '{male} and {female} are friends. Her mom is {a:prof}.',
            '{female} and {male} are friends. Her mom is {a:prof}.',
        ],
        'qas': [
            (
                'Whose mom is {a:prof}?',
                '{female}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=250,
    ))


In [48]:
show_example(t_his_her)

  [1] ('Steven and Kelly are friends. His mom is an actor.', 'Whose mom is an actor?') , pred: Steven and Kelly , label: Steven
  [2] ('Kelly and Steven are friends. His mom is an actor.', 'Whose mom is an actor?') , pred: Kelly and Steven , label: Steven
---


In [49]:
name = 'Basic coref, his / her'
test = MFT(**t_his_her, expect=expect_squad, name=name, description='', capability='Coref')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 1000 examples
Test cases:      500
Fails (rate):    500 (100.0%)

Example fails:
C: Samuel and Lucy are friends. His mom is an entrepreneur.
Q: Whose mom is an entrepreneur?
A: Samuel
P: Samuel and Lucy

C: Lucy and Samuel are friends. His mom is an entrepreneur.
Q: Whose mom is an entrepreneur?
A: Samuel
P: Lucy and Samuel


----


### 3. Former and Latter

In [50]:
t_former_latter = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} and {first_name2} are friends. The former is {a:prof1}.',
            '{first_name2} and {first_name} are friends. The latter is {a:prof1}.',
            '{first_name} and {first_name2} are friends. The former is {a:prof1} and the latter is {a:prof2}.',
            '{first_name2} and {first_name} are friends. The former is {a:prof2} and the latter is {a:prof1}.',
        ],
        'qas': [
            (
                'Who is {a:prof1}?',
                '{first_name}'
            ), 
        ]
        
    },
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))


In [51]:
show_example(t_former_latter)

  [1] ('Jean and Karen are friends. The former is an interpreter.', 'Who is an interpreter?') , pred: Jean and Karen , label: Jean
  [2] ('Karen and Jean are friends. The latter is an interpreter.', 'Who is an interpreter?') , pred: Karen and Jean are friends , label: Jean
  [3] ('Jean and Karen are friends. The former is an interpreter and the latter is an activist.', 'Who is an interpreter?') , pred: Jean and Karen , label: Jean
  [4] ('Karen and Jean are friends. The former is an activist and the latter is an interpreter.', 'Who is an interpreter?') , pred: Karen and Jean , label: Jean
---


In [52]:
name = 'Former / Latter'
test = MFT(**t_former_latter, expect=expect_squad, name=name, description='', capability='Coref')
test.run(model_predictor)
test.summary(n=1, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 1928 examples
Test cases:      482
Fails (rate):    482 (100.0%)

Example fails:
C: Alice and Edward are friends. The former is an organizer.
Q: Who is an organizer?
A: Alice
P: Alice and Edward

C: Edward and Alice are friends. The latter is an organizer.
Q: Who is an organizer?
A: Alice
P: Edward and Alice

C: Alice and Edward are friends. The former is an organizer and the latter is an architect.
Q: Who is an organizer?
A: Alice
P: Alice and Edward are friends. The former is an organizer and the latter is an architect


----


In [82]:
# suite.summary(n=0)

In [97]:
df_suite=get_summary(suite)

In [99]:
display_and_export_mdtable(df_suite, do_export=False)

| Test Name                                          |   Total Cases |   Example Per Case |   Failures | Failure Rate   |
|----------------------------------------------------|---------------|--------------------|------------|----------------|
| Comparisons                                        |           498 |                  2 |        498 | 100.00%        |
| Intensifiers                                       |           499 |                 12 |        499 | 100.00%        |
| Properites                                         |           500 |                  4 |        499 | 99.80%         |
| Profession vs nationality                          |           500 |                 10 |        260 | 52.00%         |
| Animal vs Vehicle                                  |           500 |                  4 |        495 | 99.00%         |
| Animal vs Vehicle v2                               |           499 |                  4 |        266 | 53.31%         |
| Synonyms                                           |           458 |                  4 |         21 | 4.59%          |
| Antonyms                                           |           498 |                  4 |        498 | 100.00%        |
| Antonyms Comparison                                |           499 |                 16 |        499 | 100.00%        |
| There was a change in profession                   |           485 |                  2 |         10 | 2.06%          |
| Time Difference                                    |           497 |                  4 |        497 | 100.00%        |
| Negation in context, may or may not be in question |           495 |                  4 |        487 | 98.38%         |
| Negation in question only.                         |           489 |                  8 |        489 | 100.00%        |
| Basic coref, he / she                              |           476 |                  8 |        476 | 100.00%        |
| Basic coref, his / her                             |           500 |                  2 |        500 | 100.00%        |
| Former / Latter                                    |           482 |                  4 |        482 | 100.00%        |

In [54]:
export_suite_to_jsonl(suite)

✅ Saved 45374 rows to {output_file}.
